In [2]:
import pandas as pd

In [35]:
def build_path_to_device_map(data_csv):
    path_to_device = {}
    exam_to_device = {}

    # Read the CSV file
    data = pd.read_csv(data_csv)

    # Iterate over each row in the CSV file
    for index, row in data.iterrows():
        manufacturer_model = row['device_name']
        file = row['file_path']
        view = row['view']
        patient_id = row['patient_id']
        exam_index = row['exam_id']
        exam_id = f"{patient_id} {exam_index}"

        # Modify the device name if needed
        device_name = '{} {}'.format(manufacturer_model, 'C-View') if 'C-View' in view else manufacturer_model

        # Build the path_to_device dictionary
        path_to_device[file] = device_name

        # Build the exam_to_device dictionary
        if exam_id not in exam_to_device:
            exam_to_device[exam_id] = device_name

    return path_to_device, exam_to_device


In [36]:
build_path_to_device_map('/sc/arion/projects/shenl03_ml/2023_adriana_bc_risk/Mirai_test/Mirai/data/mssm_mammogram_metadata_for_mirai_w_device.csv')

({'/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00001955.png': 'GE',
  '/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00001952.png': 'GE',
  '/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00001953.png': 'GE',
  '/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00001951.png': 'GE',
  '/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00001954.png': 'GE',
  '/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00008769.png': 'GE',
  '/sc/arion/projects/Nestlerlab/shenl03_ml/2023_adriana_bc_risk/Mirai_test/mssm_mammography/images_png_dcmtk_2048x1664/00008770.png': 'GE',
  '/sc/arion/

In [37]:
dataset = '/sc/arion/projects/shenl03_ml/2023_adriana_bc_risk/Mirai_test/Mirai/data/mssm_mammogram_metadata_for_mirai_w_device.csv'

In [38]:
def add_device_to_dataset(self):
    path_to_device, exam_to_device = self.build_path_to_device_map()
    for d in self.dataset:

        paths = [d['file_path']] if 'file_path' in d else d['paths']
        d['device_name'], d['device'], d['device_is_known'] = [], [], []

        for path in paths:
            device = path_to_device[path]
            device_id = DEVICE_TO_ID[device] if device in DEVICE_TO_ID else 0
            device_is_known = device in DEVICE_TO_ID

            d['device_name'].append(device.replace(' ', '_') if device is not None else "<UNK>")
            d['device'].append(device_id)
            d['device_is_known'].append(device_is_known)

        single_image = len(paths) == 1
        if single_image:
            d['device_name'] = d['device_name'][0]
            d['device'] = d['device'][0]
            d['device_is_known'] = d['device_is_known'][0]
        else:
            d['device_name'] = np.array(d['device_name'])
            d['device'] = np.array(d['device'])
            d['device_is_known'] = np.array(d['device_is_known'], dtype=int)

    device_dist = Counter([ d['device'] if single_image else d['device'][-1] for d in self.dataset])
    print("Device Dist: {}".format(device_dist))
    if self.split_group == 'train':
        device_count = list(device_dist.values())
        self.args.device_entropy = entropy(device_count)
        print("Device Entropy: {}".format(self.args.device_entropy))